<a href="https://colab.research.google.com/github/sowmyakavali/Traffic-Sign-Board-Detection/blob/master/Street_Light_Data_Collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/Street Light Detection/")
os.listdir()

['class-descriptions-boxable.csv',
 'train-annotations-bbox.csv',
 'train-images-boxable-with-rotation.csv',
 '.ipynb_checkpoints',
 'Street light',
 'labels2',
 'generate_label_map.py',
 'rfcn_resnet101.ipynb',
 'Street Light Detection.ipynb']

In [ ]:
import matplotlib.pyplot as plt
import cv2 as cv
from google.colab.patches import cv2_imshow
import pandas as pd
import math
from tqdm import tqdm

annos = pd.read_csv("train-annotations-bbox.csv")

In [ ]:
# !wget https://storage.googleapis.com/openimages/2018_04/class-descriptions-boxable.csv
# !wget https://storage.googleapis.com/openimages/2018_04/train/train-annotations-bbox.csv
# !wget https://storage.googleapis.com/openimages/2018_04/train/train-images-boxable-with-rotation.csv

In [ ]:
# !python3 getDataFromOpenImages.py

In [ ]:
# Visualize Data

for i in os.listdir("Street light/train")[10:12]:
  img = cv.imread("Street light/train/"+i, 1)
  width = img.shape[1]
  height = img.shape[0]
  anno = annos[annos["ImageID"] == i.split(".jpg")[0]][annos["LabelName"] == "/m/033rq4"]
  for i,j in anno.iterrows():
    xmin = math.floor(j['XMin'] * width) - 100
    ymin = math.floor(j['YMin'] * height) - 100
    xmax = math.floor(j['XMax'] * width) - 100
    ymax = math.floor(j['YMax'] * height) - 100
    img = cv.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 0, 255), 1)
    img = cv.putText(img, "Street light", (20+xmin, 20+ymin), 1, 1, (0, 0, 255))
  cv2_imshow(img)

In [ ]:
data = pd.read_csv("Street light/train_labels.csv")
exceed = []
for _,i in data.iterrows():
  print(i)
  if (i['xmin'] > i['width'] or i['xmax'] > i['width'] or i['ymin'] > i['height'] or i['ymax'] > i['height']):
    exceed.append(i)
print(len(exceed))

In [ ]:
# Generate train_labels.csv

from tqdm import tqdm

addons = []
objData = []
for i in tqdm(os.listdir("Street light/train")):
  filename = i
  img = cv.imread("Street light/train/"+i, 1)
  width = img.shape[1]
  height = img.shape[0]
  anno = annos[annos["ImageID"] == i.split(".jpg")[0]][annos["LabelName"] == "/m/033rq4"]
  for i,j in anno.iterrows():
    xmin = math.floor(j['XMin'] * width) - 100
    ymin = math.floor(j['YMin'] * height) - 100
    xmax = math.floor(j['XMax'] * width) - 100
    ymax = math.floor(j['YMax'] * height) - 100
    if (xmin >= width or ymin >= height or xmax > width or ymax > height):
      print(i)
      addons.append(i)
    objData.append([filename, width, height, "Street light", xmin, ymin, xmax, ymax])
pd.DataFrame(objData, columns = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']).to_csv('Street light/train_labels.csv', index=False)

In [ ]:
addons = []
objData = []
for i in tqdm(os.listdir("Street light/test")):
  filename = i
  img = cv.imread("Street light/test/"+i, 1)
  width = img.shape[1]
  height = img.shape[0]
  anno = annos[annos["ImageID"] == i.split(".jpg")[0]][annos["LabelName"] == "/m/033rq4"]
  for i,j in anno.iterrows():
    xmin = math.floor(j['XMin'] * width) - 10
    ymin = math.floor(j['YMin'] * height) - 10
    xmax = math.floor(j['XMax'] * width) - 10
    ymax = math.floor(j['YMax'] * height) - 10
    if (xmin >= width or ymin >= height or xmax > width or ymax > height):
      print(i)
      addons.append(i)
    objData.append([filename, width, height, "Street light", xmin, ymin, xmax, ymax])
pd.DataFrame(objData, columns = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']).to_csv('Street light/test_labels.csv', index=False)

In [ ]:
# Generate valid_labels.csv

addons = []
objData = []
weird = []
for i in tqdm(os.listdir("Street light/validate")):
  filename = i
  img = cv.imread("Street light/validate/"+i, 1)
  width = img.shape[1]
  height = img.shape[0]
  anno = annos[annos["ImageID"] == i.split(".jpg")[0]][annos["LabelName"] == "/m/033rq4"]
  for i,j in anno.iterrows():
    xmin = math.floor(j['XMin'] * width) - 10
    ymin = math.floor(j['YMin'] * height) - 10
    xmax = math.floor(j['XMax'] * width) - 10
    ymax = math.floor(j['YMax'] * height) - 10
    objData.append([filename, width, height, "Street light", xmin, ymin, xmax, ymax])
pd.DataFrame(objData, columns = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']).to_csv('Street light/validate_labels.csv', index=False)